<a href="https://colab.research.google.com/github/melchilegion/Qwasar.io/blob/main/atari_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETTING UP THE ENVIRONMENT**

In [1]:
!pip install gym[atari] tensorflow keras opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00


# **IMPORT LIBRARIES**

In [2]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


# **BUILDING THE DQN MODELS**

In [3]:
def build_model(state_size, action_size):
    model = keras.Sequential()
    model.add(layers.Dense(24, input_dim=state_size, activation='relu'))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001))
    return model

# **IMPLEMENTING THE DQN ALGORITHM**

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = build_model(state_size, action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# **TRAINING THE MODEL FOR EACH GAME**

# **CartPole**

In [5]:
def train_cartpole():
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    episodes = 1000
    batch_size = 32

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"Episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

# **Space Invaders**

In [6]:
def train_space_invaders():
    env = gym.make('SpaceInvaders-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    episodes = 1000
    batch_size = 32

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"Episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

# **Pacman**

In [7]:
def train_pacman():
    env = gym.make('MsPacman-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    episodes = 1000
    batch_size = 32

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"Episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

In [8]:
def train_cartpole():
    env = gym.make('CartPole-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size) # Make sure DQNAgent is defined
    episodes = 1000
    batch_size = 32
    scores = []

    for e in range(episodes):
        state = env.reset()

        # If state is a tuple, try reshaping only the first element
        if isinstance(state, tuple):
            state = np.reshape(state[0], [1, state_size])
        else:
            state = np.reshape(state, [1, state_size])

        score = 0
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10

            if isinstance(next_state, tuple):
                next_state = np.reshape(next_state[0], [1, state_size])
            else:
                next_state = np.reshape(next_state, [1, state_size])

            agent.remember(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                print(f"Episode: {e}/{episodes}, score: {score}, e: {agent.epsilon:.2}")
                scores.append(score)
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

    return scores

In [9]:
def train_space_invaders():
    env = gym.make('SpaceInvaders-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size) # Make sure DQNAgent is defined
    episodes = 1000
    batch_size = 32
    scores = []

    for e in range(episodes):
        state = env.reset()

        # If state is a tuple, try reshaping only the first element
        if isinstance(state, tuple):
            state = np.reshape(state[0], [1, state_size])
        else:
            state = np.reshape(state, [1, state_size])

        score = 0
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10

            if isinstance(next_state, tuple):
                next_state = np.reshape(next_state[0], [1, state_size])
            else:
                next_state = np.reshape(next_state, [1, state_size])

            agent.remember(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                print(f"Episode: {e}/{episodes}, score: {score}, e: {agent.epsilon:.2}")
                scores.append(score)
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

    return scores

In [10]:
def train_pacman():
    env = gym.make('MsPacman-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size) # Make sure DQNAgent is defined
    episodes = 1000
    batch_size = 32
    scores = []

    for e in range(episodes):
        state = env.reset()

        # If state is a tuple, try reshaping only the first element
        if isinstance(state, tuple):
            state = np.reshape(state[0], [1, state_size])
        else:
            state = np.reshape(state, [1, state_size])

        score = 0
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10

            if isinstance(next_state, tuple):
                next_state = np.reshape(next_state[0], [1, state_size])
            else:
                next_state = np.reshape(next_state, [1, state_size])

            agent.remember(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                print(f"Episode: {e}/{episodes}, score: {score}, e: {agent.epsilon:.2}")
                scores.append(score)
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

    return scores

# **VISUALIZING RESULTS**

In [ ]:
def plot_scores(scores, title):
    plt.plot(scores)
    plt.title(title)
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.show()
    plt.savefig(f"{title}.png")
    plt.close()
    return scores

if __name__ == "__main__":
    scores = train_cartpole()
    plot_scores(scores, "CartPole")
    scores = train_space_invaders()
    plot_scores(scores, "SpaceInvaders")
    scores = train_pacman()
    plot_scores(scores, "Pacman")
    plt.plot(scores)
    plt.title("Pacman")
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.savefig("Pacman.png")
    plt.close()
    plt.plot(scores)
    plt.title("SpaceInvaders")
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.savefig("SpaceInvaders.png")
    plt.close()
    plt.plot(scores)
    plt.title("CartPole")
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.savefig("CartPole.png")
    plt.close()
    plt.plot(scores)
    plt.title("Pacman")
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.show(scores)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(

Episode: 0/1000, score: 10.0, e: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━